##### Copyright 2019 The TensorFlow Authors.

In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Keras overview

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/guide/keras/overview"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/guide/keras/overview.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/guide/keras/overview.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/guide/keras/overview.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

This guide gives you the basics to get started with Keras. It's a 10-minute read.

## Import tf.keras

`tf.keras` is TensorFlow's implementation of the
[Keras API specification](https://keras.io). This is a high-level
API to build and train models that includes first-class support for
TensorFlow-specific functionality, such as [eager execution](#eager_execution),
`tf.data` pipelines, and [Estimators](../estimator.ipynb).
`tf.keras` makes TensorFlow easier to use without sacrificing flexibility and
performance.

To get started, import `tf.keras` as part of your TensorFlow program setup:

In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras

`tf.keras` can run any Keras-compatible code, but keep in mind:

* The `tf.keras` version in the latest TensorFlow release might not be the same
  as the latest `keras` version from PyPI. Check `tf.keras.__version__`.
* When [saving a model's weights](#weights_only), `tf.keras` defaults to the
  [checkpoint format](../checkpoint.ipynb). Pass `save_format='h5'` to
  use HDF5 (or pass a filename that ends in `.h5`).

## Build a simple model

### Sequential model

In Keras, you assemble *layers* to build *models*. A model is (usually) a graph
of layers. The most common type of model is a stack of layers: the
`tf.keras.Sequential` model.

To build a simple, fully-connected network (i.e. multi-layer perceptron):

In [3]:
from tensorflow.keras import layers

model = tf.keras.Sequential()
# Adds a densely-connected layer with 64 units to the model:
model.add(layers.Dense(64, activation='relu'))
# Add another:
model.add(layers.Dense(64, activation='relu'))
# Add a softmax layer with 10 output units:
model.add(layers.Dense(10, activation='softmax'))

You can find a complete, short example of how to use Sequential models [here](https://www.tensorflow.org/tutorials/quickstart/beginner).

To learn about building more advanced models than Sequential models, see:
- [Guide to the Keras Functional API](./functional.ipynb)
- [Guide to writing layers and models from scratch with subclassing](./custom_layers_and_models.ipynb)

### Configure the layers

There are many `tf.keras.layers` available. Most of them share some common constructor
arguments:

* `activation`: Set the activation function for the layer. This parameter is
  specified by the name of a built-in function or as a callable object. By
  default, no activation is applied.
* `kernel_initializer` and `bias_initializer`: The initialization schemes
  that create the layer's weights (kernel and bias). This parameter is a name or
  a callable object. This defaults to the `"Glorot uniform"` initializer.
* `kernel_regularizer` and `bias_regularizer`: The regularization schemes
  that apply the layer's weights (kernel and bias), such as L1 or L2
  regularization. By default, no regularization is applied.

The following instantiates `tf.keras.layers.Dense` layers using constructor
arguments:

In [4]:
# Create a sigmoid layer:
layers.Dense(64, activation='sigmoid')
# Or:
layers.Dense(64, activation=tf.keras.activations.sigmoid)

# A linear layer with L1 regularization of factor 0.01 applied to the kernel matrix:
layers.Dense(64, kernel_regularizer=tf.keras.regularizers.l1(0.01))

# A linear layer with L2 regularization of factor 0.01 applied to the bias vector:
layers.Dense(64, bias_regularizer=tf.keras.regularizers.l2(0.01))

# A linear layer with a kernel initialized to a random orthogonal matrix:
layers.Dense(64, kernel_initializer='orthogonal')

# A linear layer with a bias vector initialized to 2.0s:
layers.Dense(64, bias_initializer=tf.keras.initializers.Constant(2.0))

## Train and evaluate

### Set up training

After the model is constructed, configure its learning process by calling the
`compile` method:

In [5]:
model = tf.keras.Sequential([
# Adds a densely-connected layer with 64 units to the model:
layers.Dense(64, activation='relu', input_shape=(32,)),
# Add another:
layers.Dense(64, activation='relu'),
# Add a softmax layer with 10 output units:
layers.Dense(10, activation='softmax')])

model.compile(optimizer=tf.keras.optimizers.Adam(0.01),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

`tf.keras.Model.compile` takes three important arguments:

* `optimizer`: This object specifies the training procedure. Pass it optimizer
  instances from the `tf.keras.optimizers` module, such as
  `tf.keras.optimizers.Adam` or
  `tf.keras.optimizers.SGD`. If you just want to use the default parameters, you can also specify optimizers via strings, such as `'adam'` or `'sgd'`.
* `loss`: The function to minimize during optimization. Common choices include
  mean square error (`mse`), `categorical_crossentropy`, and
  `binary_crossentropy`. Loss functions are specified by name or by
  passing a callable object from the `tf.keras.losses` module.
* `metrics`: Used to monitor training. These are string names or callables from
  the `tf.keras.metrics` module.
* Additionally, to make sure the model trains and evaluates eagerly, you can make sure to pass `run_eagerly=True` as a parameter to compile.


The following shows a few examples of configuring a model for training:

In [6]:
# Configure a model for mean-squared error regression.
model.compile(optimizer=tf.keras.optimizers.Adam(0.01),
              loss='mse',       # mean squared error
              metrics=['mae'])  # mean absolute error

# Configure a model for categorical classification.
model.compile(optimizer=tf.keras.optimizers.RMSprop(0.01),
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=[tf.keras.metrics.CategoricalAccuracy()])

### Train from NumPy data

For small datasets, use in-memory [NumPy](https://www.numpy.org/)
arrays to train and evaluate a model. The model is "fit" to the training data
using the `fit` method:

In [7]:
import numpy as np

data = np.random.random((1000, 32))
labels = np.random.random((1000, 10))

model.fit(data, labels, epochs=10, batch_size=32)

Train on 1000 samples
Epoch 1/10
1000/1000 [==============================] - 1s 960us/sample - loss: 225.3230 - categorical_accuracy: 0.0910
Epoch 2/10
1000/1000 [==============================] - 0s 94us/sample - loss: 861.7445 - categorical_accuracy: 0.1000
Epoch 3/10
1000/1000 [==============================] - 0s 98us/sample - loss: 1727.2429 - categorical_accuracy: 0.1050
Epoch 4/10
1000/1000 [==============================] - 0s 103us/sample - loss: 2973.7242 - categorical_accuracy: 0.1110
Epoch 5/10
1000/1000 [==============================] - 0s 98us/sample - loss: 4355.6231 - categorical_accuracy: 0.1080
Epoch 6/10
1000/1000 [==============================] - 0s 96us/sample - loss: 6121.8853 - categorical_accuracy: 0.0980
Epoch 7/10
1000/1000 [==============================] - 0s 99us/sample - loss: 7931.4600 - categorical_accuracy: 0.1080
Epoch 8/10
1000/1000 [==============================] - 0s 101us/sample - loss: 10325.2313 - categorical_accuracy: 0.1040
Epoch 9/10
1000/

`tf.keras.Model.fit` takes three important arguments:

* `epochs`: Training is structured into *epochs*. An epoch is one iteration over
  the entire input data (this is done in smaller batches).
* `batch_size`: When passed NumPy data, the model slices the data into smaller
  batches and iterates over these batches during training. This integer
  specifies the size of each batch. Be aware that the last batch may be smaller
  if the total number of samples is not divisible by the batch size.
* `validation_data`: When prototyping a model, you want to easily monitor its
  performance on some validation data. Passing this argument—a tuple of inputs
  and labels—allows the model to display the loss and metrics in inference mode
  for the passed data, at the end of each epoch.

Here's an example using `validation_data`:

In [8]:
import numpy as np

data = np.random.random((1000, 32))
labels = np.random.random((1000, 10))

val_data = np.random.random((100, 32))
val_labels = np.random.random((100, 10))

model.fit(data, labels, epochs=10, batch_size=32,
          validation_data=(val_data, val_labels))

Train on 1000 samples, validate on 100 samples
Epoch 1/10
1000/1000 [==============================] - 0s 222us/sample - loss: 18111.9893 - categorical_accuracy: 0.1160 - val_loss: 23122.5841 - val_categorical_accuracy: 0.0900
Epoch 2/10
1000/1000 [==============================] - 0s 121us/sample - loss: 21908.4192 - categorical_accuracy: 0.1080 - val_loss: 29875.1391 - val_categorical_accuracy: 0.1400
Epoch 3/10
1000/1000 [==============================] - 0s 118us/sample - loss: 25155.9633 - categorical_accuracy: 0.0960 - val_loss: 22650.7442 - val_categorical_accuracy: 0.0900
Epoch 4/10
1000/1000 [==============================] - 0s 120us/sample - loss: 28609.6988 - categorical_accuracy: 0.0940 - val_loss: 40423.3028 - val_categorical_accuracy: 0.1400
Epoch 5/10
1000/1000 [==============================] - 0s 122us/sample - loss: 33172.2335 - categorical_accuracy: 0.1020 - val_loss: 53535.4923 - val_categorical_accuracy: 0.1400
Epoch 6/10
1000/1000 [==============================]

### Train from tf.data datasets

Use the [Datasets API](../data.ipynb) to scale to large datasets
or multi-device training. Pass a `tf.data.Dataset` instance to the `fit`
method:

In [9]:
# Instantiates a toy dataset instance:
dataset = tf.data.Dataset.from_tensor_slices((data, labels))
dataset = dataset.batch(32)

model.fit(dataset, epochs=10)

Epoch 1/10
32/32 [==============================] - 0s 4ms/step - loss: 62072.2119 - categorical_accuracy: 0.0870
Epoch 2/10
32/32 [==============================] - 0s 3ms/step - loss: 65683.7116 - categorical_accuracy: 0.1110
Epoch 3/10
32/32 [==============================] - 0s 3ms/step - loss: 72574.1263 - categorical_accuracy: 0.0940
Epoch 4/10
32/32 [==============================] - 0s 3ms/step - loss: 81504.8633 - categorical_accuracy: 0.0820
Epoch 5/10
32/32 [==============================] - 0s 3ms/step - loss: 83245.1799 - categorical_accuracy: 0.1040
Epoch 6/10
32/32 [==============================] - 0s 3ms/step - loss: 92865.8246 - categorical_accuracy: 0.0970
Epoch 7/10
32/32 [==============================] - 0s 3ms/step - loss: 100415.4726 - categorical_accuracy: 0.0950
Epoch 8/10
32/32 [==============================] - 0s 3ms/step - loss: 103184.3824 - categorical_accuracy: 0.1130
Epoch 9/10
32/32 [==============================] - 0s 3ms/step - loss: 114717.4216 - 

Since the `Dataset` yields batches of data, this snippet does not require a `batch_size`.

Datasets can also be used for validation:

In [10]:
dataset = tf.data.Dataset.from_tensor_slices((data, labels))
dataset = dataset.batch(32)

val_dataset = tf.data.Dataset.from_tensor_slices((val_data, val_labels))
val_dataset = val_dataset.batch(32)

model.fit(dataset, epochs=10,
          validation_data=val_dataset)

Epoch 1/10
32/32 [==============================] - 0s 4ms/step - loss: 128715.9932 - categorical_accuracy: 0.0920 - val_loss: 0.0000e+00 - val_categorical_accuracy: 0.0000e+00
Epoch 2/10
32/32 [==============================] - 0s 4ms/step - loss: 134195.9842 - categorical_accuracy: 0.0940 - val_loss: 202330.3516 - val_categorical_accuracy: 0.1000
Epoch 3/10
32/32 [==============================] - 0s 4ms/step - loss: 149120.2739 - categorical_accuracy: 0.0850 - val_loss: 239164.6758 - val_categorical_accuracy: 0.1400
Epoch 4/10
32/32 [==============================] - 0s 4ms/step - loss: 154822.5747 - categorical_accuracy: 0.1180 - val_loss: 138043.9570 - val_categorical_accuracy: 0.1200
Epoch 5/10
32/32 [==============================] - 0s 4ms/step - loss: 166273.9298 - categorical_accuracy: 0.0990 - val_loss: 171081.7344 - val_categorical_accuracy: 0.0900
Epoch 6/10
32/32 [==============================] - 0s 4ms/step - loss: 172326.5566 - categorical_accuracy: 0.0960 - val_loss: 

### Evaluate and predict

The `tf.keras.Model.evaluate` and `tf.keras.Model.predict` methods can use NumPy
data and a `tf.data.Dataset`.

Here's how to *evaluate* the inference-mode loss and metrics for the data provided:

In [11]:
# With Numpy arrays
data = np.random.random((1000, 32))
labels = np.random.random((1000, 10))

model.evaluate(data, labels, batch_size=32)

# With a Dataset
dataset = tf.data.Dataset.from_tensor_slices((data, labels))
dataset = dataset.batch(32)

model.evaluate(dataset)

1000/1 [================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

[200361.98486328125, 0.099]

And here's how to *predict* the output of the last layer in inference for the data provided,
as a NumPy array:

In [12]:
result = model.predict(data, batch_size=32)
print(result.shape)

(1000, 10)


For a complete guide on training and evaluation, including how to write custom training loops from scratch, see the [guide to training and evaluation](./train_and_evaluate.ipynb).

## Build complex models

### The Functional API

 The `tf.keras.Sequential` model is a simple stack of layers that cannot
represent arbitrary models. Use the
[Keras functional API](./functional.ipynb)
to build complex model topologies such as:

* Multi-input models,
* Multi-output models,
* Models with shared layers (the same layer called several times),
* Models with non-sequential data flows (e.g. residual connections).

Building a model with the functional API works like this:

1. A layer instance is callable and returns a tensor.
2. Input tensors and output tensors are used to define a `tf.keras.Model`
   instance.
3. This model is trained just like the `Sequential` model.

The following example uses the functional API to build a simple, fully-connected
network:

In [13]:
inputs = tf.keras.Input(shape=(32,))  # Returns an input placeholder

# A layer instance is callable on a tensor, and returns a tensor.
x = layers.Dense(64, activation='relu')(inputs)
x = layers.Dense(64, activation='relu')(x)
predictions = layers.Dense(10, activation='softmax')(x)

Instantiate the model given inputs and outputs.

In [14]:
model = tf.keras.Model(inputs=inputs, outputs=predictions)

# The compile step specifies the training configuration.
model.compile(optimizer=tf.keras.optimizers.RMSprop(0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Trains for 5 epochs
model.fit(data, labels, batch_size=32, epochs=5)

Train on 1000 samples
Epoch 1/5
1000/1000 [==============================] - 0s 490us/sample - loss: 13.1229 - accuracy: 0.0930
Epoch 2/5
1000/1000 [==============================] - 0s 84us/sample - loss: 23.2754 - accuracy: 0.0970
Epoch 3/5
1000/1000 [==============================] - 0s 83us/sample - loss: 44.1900 - accuracy: 0.0920
Epoch 4/5
1000/1000 [==============================] - 0s 84us/sample - loss: 70.6432 - accuracy: 0.0980
Epoch 5/5
1000/1000 [==============================] - 0s 84us/sample - loss: 103.5022 - accuracy: 0.1040


### Model subclassing

Build a fully-customizable model by subclassing `tf.keras.Model` and defining
your own forward pass. Create layers in the `__init__` method and set them as
attributes of the class instance. Define the forward pass in the `call` method.

Model subclassing is particularly useful when
[eager execution](../eager.ipynb) is enabled, because it allows the forward pass
to be written imperatively.

Note: if you need your model to *always* run imperatively, you can set `dynamic=True` when calling the `super` constructor.

> Key Point: Use the right API for the job. While model subclassing offers
flexibility, it comes at a cost of greater complexity and more opportunities for
user errors. If possible, prefer the functional API.

The following example shows a subclassed `tf.keras.Model` using a custom forward
pass that does not have to be run imperatively:

In [15]:
class MyModel(tf.keras.Model):

  def __init__(self, num_classes=10):
    super(MyModel, self).__init__(name='my_model')
    self.num_classes = num_classes
    # Define your layers here.
    self.dense_1 = layers.Dense(32, activation='relu')
    self.dense_2 = layers.Dense(num_classes, activation='sigmoid')

  def call(self, inputs):
    # Define your forward pass here,
    # using layers you previously defined (in `__init__`).
    x = self.dense_1(inputs)
    return self.dense_2(x)

Instantiate the new model class:

In [16]:
model = MyModel(num_classes=10)

# The compile step specifies the training configuration.
model.compile(optimizer=tf.keras.optimizers.RMSprop(0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Trains for 5 epochs.
model.fit(data, labels, batch_size=32, epochs=5)

Train on 1000 samples
Epoch 1/5
1000/1000 [==============================] - 1s 594us/sample - loss: 11.6543 - accuracy: 0.1140
Epoch 2/5
1000/1000 [==============================] - 0s 87us/sample - loss: 11.6339 - accuracy: 0.1070
Epoch 3/5
1000/1000 [==============================] - 0s 85us/sample - loss: 11.6271 - accuracy: 0.1050
Epoch 4/5
1000/1000 [==============================] - 0s 84us/sample - loss: 11.6223 - accuracy: 0.1080
Epoch 5/5
1000/1000 [==============================] - 0s 85us/sample - loss: 11.6190 - accuracy: 0.1080


### Custom layers

Create a custom layer by subclassing `tf.keras.layers.Layer` and implementing
the following methods:

* `__init__`: Optionally define sublayers to be used by this layer.
* `build`: Create the weights of the layer. Add weights with the `add_weight`
  method.
* `call`: Define the forward pass.
* Optionally, a layer can be serialized by implementing the `get_config` method
  and the `from_config` class method.

Here's an example of a custom layer that implements a `matmul` of an input with
a kernel matrix:

In [17]:
class MyLayer(layers.Layer):

  def __init__(self, output_dim, **kwargs):
    self.output_dim = output_dim
    super(MyLayer, self).__init__(**kwargs)

  def build(self, input_shape):
    # Create a trainable weight variable for this layer.
    self.kernel = self.add_weight(name='kernel',
                                  shape=(input_shape[1], self.output_dim),
                                  initializer='uniform',
                                  trainable=True)

  def call(self, inputs):
    return tf.matmul(inputs, self.kernel)

  def get_config(self):
    base_config = super(MyLayer, self).get_config()
    base_config['output_dim'] = self.output_dim
    return base_config

  @classmethod
  def from_config(cls, config):
    return cls(**config)

Create a model using your custom layer:

In [18]:
model = tf.keras.Sequential([
    MyLayer(10),
    layers.Activation('softmax')])

# The compile step specifies the training configuration
model.compile(optimizer=tf.keras.optimizers.RMSprop(0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Trains for 5 epochs.
model.fit(data, labels, batch_size=32, epochs=5)

Train on 1000 samples
Epoch 1/5
1000/1000 [==============================] - 0s 293us/sample - loss: 11.6593 - accuracy: 0.1030
Epoch 2/5
1000/1000 [==============================] - 0s 65us/sample - loss: 11.6580 - accuracy: 0.1030
Epoch 3/5
1000/1000 [==============================] - 0s 63us/sample - loss: 11.6586 - accuracy: 0.1010
Epoch 4/5
1000/1000 [==============================] - 0s 63us/sample - loss: 11.6580 - accuracy: 0.1020
Epoch 5/5
1000/1000 [==============================] - 0s 61us/sample - loss: 11.6578 - accuracy: 0.1010


Learn more about creating new layers and models from scratch with subclassing in the [Guide to writing layers and models from scratch](./custom_layers_and_models.ipynb).

## Callbacks

A callback is an object passed to a model to customize and extend its behavior
during training. You can write your own custom callback, or use the built-in
`tf.keras.callbacks` that include:

* `tf.keras.callbacks.ModelCheckpoint`: Save checkpoints of your model at
  regular intervals.
* `tf.keras.callbacks.LearningRateScheduler`: Dynamically change the learning
  rate.
* `tf.keras.callbacks.EarlyStopping`: Interrupt training when validation
  performance has stopped improving.
* `tf.keras.callbacks.TensorBoard`: Monitor the model's behavior using
  [TensorBoard](https://tensorflow.org/tensorboard).

To use a `tf.keras.callbacks.Callback`, pass it to the model's `fit` method:

In [19]:
callbacks = [
  # Interrupt training if `val_loss` stops improving for over 2 epochs
  tf.keras.callbacks.EarlyStopping(patience=2, monitor='val_loss'),
  # Write TensorBoard logs to `./logs` directory
  tf.keras.callbacks.TensorBoard(log_dir='./logs')
]
model.fit(data, labels, batch_size=32, epochs=5, callbacks=callbacks,
          validation_data=(val_data, val_labels))

Train on 1000 samples, validate on 100 samples
Epoch 1/5
1000/1000 [==============================] - 0s 159us/sample - loss: 11.6576 - accuracy: 0.1030 - val_loss: 11.7590 - val_accuracy: 0.0800
Epoch 2/5
1000/1000 [==============================] - 0s 81us/sample - loss: 11.6565 - accuracy: 0.1020 - val_loss: 11.7594 - val_accuracy: 0.0800
Epoch 3/5
1000/1000 [==============================] - 0s 81us/sample - loss: 11.6566 - accuracy: 0.1030 - val_loss: 11.7578 - val_accuracy: 0.0800
Epoch 4/5
1000/1000 [==============================] - 0s 78us/sample - loss: 11.6552 - accuracy: 0.1040 - val_loss: 11.7607 - val_accuracy: 0.0800
Epoch 5/5
1000/1000 [==============================] - 0s 79us/sample - loss: 11.6568 - accuracy: 0.1030 - val_loss: 11.7568 - val_accuracy: 0.0800



## Save and restore

### Save just the weights values

Save and load the weights of a model using `tf.keras.Model.save_weights`:

In [20]:
model = tf.keras.Sequential([
layers.Dense(64, activation='relu', input_shape=(32,)),
layers.Dense(10, activation='softmax')])

model.compile(optimizer=tf.keras.optimizers.Adam(0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [21]:
# Save weights to a TensorFlow Checkpoint file
model.save_weights('./weights/my_model')

# Restore the model's state,
# this requires a model with the same architecture.
model.load_weights('./weights/my_model')

By default, this saves the model's weights in the
[TensorFlow checkpoint](../checkpoint.ipynb) file format. Weights can
also be saved to the Keras HDF5 format (the default for the multi-backend
implementation of Keras):

In [22]:
# Save weights to a HDF5 file
model.save_weights('my_model.h5', save_format='h5')

# Restore the model's state
model.load_weights('my_model.h5')

### Save just the model configuration

A model's configuration can be saved—this serializes the model architecture
without any weights. A saved configuration can recreate and initialize the same
model, even without the code that defined the original model. Keras supports
JSON and YAML serialization formats:

In [23]:
# Serialize a model to JSON format
json_string = model.to_json()
json_string

'{"class_name": "Sequential", "config": {"name": "sequential_3", "layers": [{"class_name": "Dense", "config": {"name": "dense_17", "trainable": true, "batch_input_shape": [null, 32], "dtype": "float32", "units": 64, "activation": "relu", "use_bias": true, "kernel_initializer": {"class_name": "GlorotUniform", "config": {"seed": null}}, "bias_initializer": {"class_name": "Zeros", "config": {}}, "kernel_regularizer": null, "bias_regularizer": null, "activity_regularizer": null, "kernel_constraint": null, "bias_constraint": null}}, {"class_name": "Dense", "config": {"name": "dense_18", "trainable": true, "dtype": "float32", "units": 10, "activation": "softmax", "use_bias": true, "kernel_initializer": {"class_name": "GlorotUniform", "config": {"seed": null}}, "bias_initializer": {"class_name": "Zeros", "config": {}}, "kernel_regularizer": null, "bias_regularizer": null, "activity_regularizer": null, "kernel_constraint": null, "bias_constraint": null}}]}, "keras_version": "2.2.4-tf", "backen

In [24]:
import json
import pprint
pprint.pprint(json.loads(json_string))

{'backend': 'tensorflow',
 'class_name': 'Sequential',
 'config': {'layers': [{'class_name': 'Dense',
                        'config': {'activation': 'relu',
                                   'activity_regularizer': None,
                                   'batch_input_shape': [None, 32],
                                   'bias_constraint': None,
                                   'bias_initializer': {'class_name': 'Zeros',
                                                        'config': {}},
                                   'bias_regularizer': None,
                                   'dtype': 'float32',
                                   'kernel_constraint': None,
                                   'kernel_initializer': {'class_name': 'GlorotUniform',
                                                          'config': {'seed': None}},
                                   'kernel_regularizer': None,
                                   'name': 'dense_17',
                            

Recreate the model (newly initialized) from the JSON:

In [25]:
fresh_model = tf.keras.models.model_from_json(json_string)

Serializing a model to YAML format requires that you install `pyyaml` *before you import TensorFlow*:

In [26]:
yaml_string = model.to_yaml()
print(yaml_string)

backend: tensorflow
class_name: Sequential
config:
  layers:
  - class_name: Dense
    config:
      activation: relu
      activity_regularizer: null
      batch_input_shape: !!python/tuple [null, 32]
      bias_constraint: null
      bias_initializer:
        class_name: Zeros
        config: {}
      bias_regularizer: null
      dtype: float32
      kernel_constraint: null
      kernel_initializer:
        class_name: GlorotUniform
        config: {seed: null}
      kernel_regularizer: null
      name: dense_17
      trainable: true
      units: 64
      use_bias: true
  - class_name: Dense
    config:
      activation: softmax
      activity_regularizer: null
      bias_constraint: null
      bias_initializer:
        class_name: Zeros
        config: {}
      bias_regularizer: null
      dtype: float32
      kernel_constraint: null
      kernel_initializer:
        class_name: GlorotUniform
        config: {seed: null}
      kernel_regularizer: null
      name: dense_18
      trai

Recreate the model from the YAML:

In [27]:
fresh_model = tf.keras.models.model_from_yaml(yaml_string)

Caution: Subclassed models are not serializable because their architecture is
defined by the Python code in the body of the `call` method.


### Save the entire model in one file

The entire model can be saved to a file that contains the weight values, the
model's configuration, and even the optimizer's configuration. This allows you
to checkpoint a model and resume training later—from the exact same
state—without access to the original code.

In [28]:
# Create a simple model
model = tf.keras.Sequential([
  layers.Dense(10, activation='softmax', input_shape=(32,)),
  layers.Dense(10, activation='softmax')
])
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(data, labels, batch_size=32, epochs=5)


# Save entire model to a HDF5 file
model.save('my_model.h5')

# Recreate the exact same model, including weights and optimizer.
model = tf.keras.models.load_model('my_model.h5')

Train on 1000 samples
Epoch 1/5
1000/1000 [==============================] - 0s 401us/sample - loss: 11.6421 - accuracy: 0.1110
Epoch 2/5
1000/1000 [==============================] - 0s 76us/sample - loss: 11.6515 - accuracy: 0.1110
Epoch 3/5
1000/1000 [==============================] - 0s 76us/sample - loss: 11.6566 - accuracy: 0.1100
Epoch 4/5
1000/1000 [==============================] - 0s 76us/sample - loss: 11.6574 - accuracy: 0.1080
Epoch 5/5
1000/1000 [==============================] - 0s 77us/sample - loss: 11.6618 - accuracy: 0.1110


Learn more about saving and serialization for Keras models in the guide to [save and serialize models](./save_and_serialize.ipynb).

## Eager execution

[Eager execution](../eager.ipynb) is an imperative programming
environment that evaluates operations immediately. This is not required for
Keras, but is supported by `tf.keras` and useful for inspecting your program and
debugging.

All of the `tf.keras` model-building APIs are compatible with eager execution.
And while the `Sequential` and functional APIs can be used, eager execution
especially benefits *model subclassing* and building *custom layers*—the APIs
that require you to write the forward pass as code (instead of the APIs that
create models by assembling existing layers).

See the [eager execution guide](../eager.ipynb) for
examples of using Keras models with custom training loops and `tf.GradientTape`.
You can also find a complete, short example [here](https://www.tensorflow.org/tutorials/quickstart/advanced).

## Distribution


### Multiple GPUs

`tf.keras` models can run on multiple GPUs using
`tf.distribute.Strategy`. This API provides distributed
training on multiple GPUs with almost no changes to existing code.

Currently, `tf.distribute.MirroredStrategy` is the only supported
distribution strategy. `MirroredStrategy` does in-graph replication with
synchronous training using all-reduce on a single machine. To use
`distribute.Strategy`s , nest the optimizer instantiation and model construction and compilation in a `Strategy`'s `.scope()`, then
train the model.

The following example distributes a `tf.keras.Model` across multiple GPUs on a
single machine.

First, define a model inside the distributed strategy scope:

In [29]:
strategy = tf.distribute.MirroredStrategy()

with strategy.scope():
  model = tf.keras.Sequential()
  model.add(layers.Dense(16, activation='relu', input_shape=(10,)))
  model.add(layers.Dense(1, activation='sigmoid'))

  optimizer = tf.keras.optimizers.SGD(0.2)

  model.compile(loss='binary_crossentropy', optimizer=optimizer)

model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_21 (Dense)             (None, 16)                176       
_________________________________________________________________
dense_22 (Dense)             (None, 1)                 17        
Total params: 193
Trainable params: 193
Non-trainable params: 0
_________________________________________________________________


Next, train the model on data as usual:

In [30]:
x = np.random.random((1024, 10))
y = np.random.randint(2, size=(1024, 1))
x = tf.cast(x, tf.float32)
dataset = tf.data.Dataset.from_tensor_slices((x, y))
dataset = dataset.shuffle(buffer_size=1024).batch(32)

model.fit(dataset, epochs=1)

32/32 [==============================] - 2s 75ms/step - loss: 0.7062


For more information, see the [full guide on Distributed Training in TensorFlow](../distributed_training.ipynb).